In [4]:
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
import pandas as pd



In [5]:
# Load CSV
books = pd.read_csv("books_cleaned.csv")

# Export descriptions to TXT
books["tagged_description"].to_csv(
    "tagged_description.txt",
    sep="\n", index=False, header=False
)

# Load documents
raw_documents = TextLoader("tagged_description.txt").load()

# Split lines into individual docs
text_splitter = CharacterTextSplitter(
    chunk_size=1, chunk_overlap=0, separator="\n"
)
documents = text_splitter.split_documents(raw_documents)




Created a chunk of size 1168, which is longer than the specified 1
Created a chunk of size 1214, which is longer than the specified 1
Created a chunk of size 373, which is longer than the specified 1
Created a chunk of size 309, which is longer than the specified 1
Created a chunk of size 483, which is longer than the specified 1
Created a chunk of size 482, which is longer than the specified 1
Created a chunk of size 960, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 843, which is longer than the specified 1
Created a chunk of size 296, which is longer than the specified 1
Created a chunk of size 197, which is longer than the specified 1
Created a chunk of size 881, which is longer than the specified 1
Created a chunk of size 1088, which is longer than the specified 1
Created a chunk of size 1189, which is longer than the specified 1
Created a chunk of size 304, which is longer than the specified 1
Create

In [9]:
model = SentenceTransformer("BAAI/bge-large-en-v1.5")  # → 1024 dimension


class BGE_Embeddings:
    def embed_documents(self, docs):
        return model.encode(docs, normalize_embeddings=True)

    def embed_query(self, query):

        return model.encode([query], normalize_embeddings=True)[0]




embeddings = BGE_Embeddings()

# Create vector DB
db_books = Chroma.from_documents(
    documents,
    embedding=embeddings,
    persist_directory="chroma_bge"   # <<< NEW FOLDER (prevents dimension conflict)
)



In [14]:
def retrieve_semantic_recommendations(query: str, top_k: int = 10):
    recs = db_books.similarity_search(query, k=top_k)

    books_list = []

    for r in recs:
        isbn_str = r.page_content.split()[0].strip().replace('"', '').replace("'", "")
        try:
            isbn = int(isbn_str)
            books_list.append(isbn)
        except ValueError:
            continue

    return books[books["isbn13"].isin(books_list)]

# Test:
retrieve_semantic_recommendations("A book to teach children about nature")


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,title ans subtitles,tagged_description
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,The Sense of Wonder,9780067575208 First published more than three ...
855,9780143037392,0143037390,The Read-aloud Handbook,Jim Trelease,Language Arts & Disciplines,http://books.google.com/books/content?id=B2_yU...,Explains the importance of reading aloud to ch...,2006.0,4.40,432.0,4122.0,The Read-aloud Handbook,The Read-aloud Handbook,9780143037392 Explains the importance of readi...
1247,9780312369347,0312369344,Beatrix Potter,Linda Lear,Biography & Autobiography,http://books.google.com/books/content?id=T9uTx...,"Peter Rabbit, Mr. McGregor, and many other Bea...",2007.0,4.03,584.0,1270.0,Beatrix Potter: A Life in Nature,Beatrix Potter: A Life in Nature,"9780312369347 Peter Rabbit, Mr. McGregor, and ..."
1907,9780393315110,0393315118,Uncommon Ground: Rethinking the Human Place in...,William Cronon,Law,http://books.google.com/books/content?id=w04mj...,"Essays by revisionist historians, scientists, ...",1996.0,4.16,560.0,649.0,Uncommon Ground: Rethinking the Human Place in...,Uncommon Ground: Rethinking the Human Place in...,9780393315110 Essays by revisionist historians...
3201,9780689823824,0689823827,A Child's Garden of Verses,Robert Louis Stevenson,Juvenile Fiction,http://books.google.com/books/content?id=luTZA...,"Here is a delightful look at childhood, writte...",1999.0,4.30,67.0,21780.0,A Child's Garden of Verses,A Child's Garden of Verses,9780689823824 Here is a delightful look at chi...
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...
3748,9780786808373,0786808373,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=0jxHP...,"Introducing your baby to birds, cats, dogs, an...",2002.0,3.78,20.0,9.0,Baby Einstein: Birds,Baby Einstein: Birds,"9780786808373 Introducing your baby to birds, ..."
3749,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,Baby Einstein: Babies,"9780786808380 Introduce your babies to birds, ..."
3750,9780786808397,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,Baby Einstein: Dogs,Baby Einstein: Dogs,"9780786808397 Introduce your baby to birds, ca..."
4231,9780898599596,0898599598,The Ecological Approach to Visual Perception,James Jerome Gibson,Psychology,http://books.google.com/books/content?id=DrhCC...,This is a book about how we see: the environme...,1986.0,4.28,348.0,122.0,The Ecological Approach to Visual Perception,The Ecological Approach to Visual Perception,9780898599596 This is a book about how we see:...
